In [2]:
import numpy as np
import pandas as pd
import gc

In [3]:
#导入数据
paths = './data'
data = pd.read_csv(f'{paths}/user_log_format1.csv', dtype={'time_stamp':'str'})

In [4]:
data1 = pd.read_csv(f'{paths}/user_info_format1.csv')
data2 = pd.read_csv(f'{paths}/train_format1.csv')
submission = pd.read_csv(f'{paths}/test_format1.csv')
data_train = pd.read_csv(f'{paths}/train_format2.csv')

In [5]:
#将user_log中的seller_id更改名字
data.rename(columns={'seller_id':'merchant_id'}, inplace=True)

In [6]:
#创建matrix
data2['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([data2, submission], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)
matrix = matrix.merge(data1, on='user_id', how='left')

In [7]:
#更改数据类型
data['user_id'] = data['user_id'].astype('int32')
data['merchant_id'] = data['merchant_id'].astype('int32')
data['item_id'] = data['item_id'].astype('int32')
data['cat_id'] = data['cat_id'].astype('int32')
data['brand_id'].fillna(0, inplace=True)
data['brand_id'] = data['brand_id'].astype('int32')
data['time_stamp'] = pd.to_datetime(data['time_stamp'], format='%H%M')



In [8]:
#更改matrix中的数据类型
matrix['age_range'].fillna(0, inplace=True)
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

del data1, data2
gc.collect()   #回收资源

35

In [9]:
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6,0
1,34176,121,0.0,train,6,0
2,34176,4356,1.0,train,6,0
3,34176,2217,0.0,train,6,0
4,230784,4818,0.0,train,0,0
...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0
522337,97919,2341,nan,test,8,1
522338,97919,3971,nan,test,8,1
522339,32639,3536,nan,test,0,0


In [10]:
#以user_id分组组合
groups = data.groupby(['user_id','merchant_id'])

In [11]:
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6,0
1,34176,121,0.0,train,6,0
2,34176,4356,1.0,train,6,0
3,34176,2217,0.0,train,6,0
4,230784,4818,0.0,train,0,0
...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0
522337,97919,2341,nan,test,8,1
522338,97919,3971,nan,test,8,1
522339,32639,3536,nan,test,0,0


In [12]:
#提取用户在每个商家的行为信息,用临时变量temp保存
temp = groups['action_type'].value_counts().unstack()
temp = temp.reset_index()
temp.rename(columns = {0:'act_click',1:'act_cart',2:'act_buy',3:'act_collection'},inplace=True)
print(temp.head(10))

action_type  user_id  merchant_id  act_click  act_cart  act_buy  \
0                  1          471        1.0       NaN      NaN   
1                  1          739        1.0       NaN      NaN   
2                  1          925        3.0       NaN      1.0   
3                  1         1019       10.0       NaN      4.0   
4                  1         1156        1.0       NaN      NaN   
5                  1         2245        5.0       NaN      NaN   
6                  1         4026        4.0       NaN      1.0   
7                  1         4177        1.0       NaN      NaN   
8                  1         4335        1.0       NaN      NaN   
9                  2          420       23.0       NaN      3.0   

action_type  act_collection  
0                       NaN  
1                       NaN  
2                       NaN  
3                       NaN  
4                       NaN  
5                       NaN  
6                       NaN  
7                     

In [13]:
#将temp和matrix，以user_id,merchant_id进行组合
matrix = pd.merge(matrix,temp,on=['user_id','merchant_id'])

In [14]:
matrix  #查看数据

,user_id,merchant_id,label,origin,age_range,gender,act_click,act_cart,act_buy,act_collection
0,34176,3906,0.0,train,6,0,36.0,NaN,1.0,2.0
1,34176,121,0.0,train,6,0,13.0,NaN,1.0,NaN
2,34176,4356,1.0,train,6,0,12.0,NaN,6.0,NaN
3,34176,2217,0.0,train,6,0,1.0,NaN,1.0,NaN
4,230784,4818,0.0,train,0,0,7.0,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,4.0,NaN,1.0,NaN
522337,97919,2341,nan,test,8,1,1.0,NaN,1.0,NaN
522338,97919,3971,nan,test,8,1,12.0,NaN,4.0,NaN
522339,32639,3536,nan,test,0,0,2.0,NaN,1.0,NaN


In [15]:
matrix['act_click'].fillna(0, inplace=True)
matrix['act_cart'].fillna(0, inplace=True)
matrix['act_buy'].fillna(0, inplace=True)
matrix['act_collection'].fillna(0, inplace=True)

In [16]:
matrix

,user_id,merchant_id,label,origin,age_range,gender,act_click,act_cart,act_buy,act_collection
0,34176,3906,0.0,train,6,0,36.0,0.0,1.0,2.0
1,34176,121,0.0,train,6,0,13.0,0.0,1.0,0.0
2,34176,4356,1.0,train,6,0,12.0,0.0,6.0,0.0
3,34176,2217,0.0,train,6,0,1.0,0.0,1.0,0.0
4,230784,4818,0.0,train,0,0,7.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,4.0,0.0,1.0,0.0
522337,97919,2341,nan,test,8,1,1.0,0.0,1.0,0.0
522338,97919,3971,nan,test,8,1,12.0,0.0,4.0,0.0
522339,32639,3536,nan,test,0,0,2.0,0.0,1.0,0.0


In [17]:
#先进行导出
matrix.to_csv('./new_train.csv', index=False)

In [18]:
###加入用户喜欢逛的商家（特征）
#按照user_id进行分组
groups = data.groupby(['user_id'])
#提取每个用户喜欢逛的商家
temp = groups['merchant_id'].value_counts()
temp.rename("Sca_mer_count",inplace=True)
temp = temp.reset_index()
print(temp.head(10))

   user_id  merchant_id  Sca_mer_count
0        1         1019             14
1        1         2245              5
2        1         4026              5
3        1          925              4
4        1          471              1
5        1          739              1
6        1         1156              1
7        1         4177              1
8        1         4335              1
9        2          420             26


In [19]:
#实验测试merge能否匹配两个键
matrix = pd.merge(matrix,temp,on=['user_id','merchant_id'])

In [20]:
matrix #匹配成功

,user_id,merchant_id,label,origin,age_range,gender,act_click,act_cart,act_buy,act_collection,Sca_mer_count
0,34176,3906,0.0,train,6,0,36.0,0.0,1.0,2.0,39
1,34176,121,0.0,train,6,0,13.0,0.0,1.0,0.0,14
2,34176,4356,1.0,train,6,0,12.0,0.0,6.0,0.0,18
3,34176,2217,0.0,train,6,0,1.0,0.0,1.0,0.0,2
4,230784,4818,0.0,train,0,0,7.0,0.0,1.0,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,4.0,0.0,1.0,0.0,5
522337,97919,2341,nan,test,8,1,1.0,0.0,1.0,0.0,2
522338,97919,3971,nan,test,8,1,12.0,0.0,4.0,0.0,16
522339,32639,3536,nan,test,0,0,2.0,0.0,1.0,0.0,3


In [21]:
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

In [22]:
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

del temp, matrix
gc.collect()

20

In [23]:
#导入分析库
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report

In [24]:
import xgboost as xgb

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.3)

In [26]:
model = xgb.XGBClassifier(
    max_depth=13,
    n_estimators=1000,
    min_child_weight=250, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=1.5,    
    seed=42
    
)

In [27]:
model.fit(
    X_train, 
    y_train,
    eval_metric='auc',
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    early_stopping_rounds=10
)

[0]	validation_0-auc:0.60536	validation_1-auc:0.60110
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.61521	validation_1-auc:0.60514
[2]	validation_0-auc:0.61872	validation_1-auc:0.60246
[3]	validation_0-auc:0.62480	validation_1-auc:0.61123
[4]	validation_0-auc:0.62597	validation_1-auc:0.60529
[5]	validation_0-auc:0.63244	validation_1-auc:0.61003
[6]	validation_0-auc:0.63833	validation_1-auc:0.61426
[7]	validation_0-auc:0.64119	validation_1-auc:0.61428
[8]	validation_0-auc:0.64344	validation_1-auc:0.61573
[9]	validation_0-auc:0.64503	validation_1-auc:0.61651
[10]	validation_0-auc:0.64875	validation_1-auc:0.61976
[11]	validation_0-auc:0.65023	validation_1-auc:0.61961
[12]	validation_0-auc:0.65189	validation_1-auc:0.62079
[13]	validation_0-auc:0.65370	validation_1-auc:0.62082
[14]	validation_0-auc:0.65387	validation_1-auc:0.61802
[15]	validation_0-auc:0.6564

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=1.5, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=1.5, max_delta_step=0, max_depth=13,
              min_child_weight=250, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)

In [28]:
prob = model.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('./submission.csv', index=False)

In [144]:
#商品购买次数提取
groups = data.groupby(['user_id','merchant_id'])
#按照user_id和	merchant_id进行分组
#提取每个用户在每家购买次数
temp = groups['action_type'].value_counts()
temp.rename("action_buy_count",inplace=True)
temp = temp.reset_index()

In [145]:

temp = temp[temp['action_type']==2]
print(temp.head(10))

    user_id  merchant_id  action_type  action_buy_count
3         1          925            2                 1
5         1         1019            2                 4
9         1         4026            2                 1
13        2          420            2                 3
14        2         1179            2                 1
16        2         1679            2                 3
17        2         1784            2                 2
20        2         1974            2                 1
21        2         2076            2                 1
25        2         3552            2                 2


In [173]:
matrix = pd.read_csv(f'{paths}/new_train.csv')

In [153]:
type(matrix)

pandas.core.frame.DataFrame

In [175]:
#########
#新增一列用于存储用户点击购买比
matrix['B_t_C']=0

In [176]:
matrix['B_t_C'] = matrix['act_buy']/matrix['act_click']

In [181]:
matrix = matrix.replace([np.inf, -np.inf], np.nan)

In [184]:
matrix['B_t_C'].fillna(0, inplace=True)

In [185]:
matrix

,user_id,merchant_id,label,origin,age_range,gender,act_click,act_cart,act_buy,act_collection,B_t_C
0,34176,3906,0.0,train,6,0,36.0,0.0,1.0,2.0,0.027778
1,34176,121,0.0,train,6,0,13.0,0.0,1.0,0.0,0.076923
2,34176,4356,1.0,train,6,0,12.0,0.0,6.0,0.0,0.500000
3,34176,2217,0.0,train,6,0,1.0,0.0,1.0,0.0,1.000000
4,230784,4818,0.0,train,0,0,7.0,0.0,1.0,0.0,0.142857
5,362112,2618,0.0,train,4,1,0.0,0.0,1.0,0.0,0.000000
6,34944,2051,0.0,train,5,0,2.0,0.0,1.0,0.0,0.500000
7,231552,3828,1.0,train,5,0,78.0,0.0,5.0,0.0,0.064103
8,231552,2124,0.0,train,5,0,6.0,0.0,1.0,0.0,0.166667
9,232320,1168,0.0,train,4,1,2.0,0.0,1.0,1.0,0.500000
